In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Planning

### What data do we have?
- Everyone's picks and weights for n weeks
- The winners of every game

### League average pick
- Calculate a league average pick: an "expected estimate" of what the rest of the league will pick
    - Expected that the league average pick is closer to 0 than the actual score
    - Always positive -> just in the direction of the most picked team
    - Ranges from 0 - 16 
    - Closer to 0: league is pretty split
    - Closer to 16: league favoring one team
    
### League average Score
- Calculate a league average score: The actual average score for each game in the league
    - Ranges from 0 - 16
    - This is just the average number of points that the league gained
    - 

In [57]:
def html_to_df(week):
    filepath = f"data/week{week}.html"

    
    # na, correct, incorrect
    # week, game, correct points, incorrect points
    data = []

    with open(filepath, "r") as file:
        contents = file.read()
        picks_length = 16 # need to change for other weeks
        soup = BeautifulSoup(contents)

        rows = soup.find_all("tr", attrs={"class": "bg2"})
        person = 0
        for row in rows:
            picks = row.find_all("td", attrs={"class": ["na", "correct", "incorrect"]})
            if len(picks) != picks_length:
                continue
            i = 0
            for pick in picks:
                pick_class = pick["class"][0]
                if pick_class == "na":
                    #data.append([week, i, person, 0, 0])
                    i += 1
                    continue

                pick_text = pick.text
                pts = pick_text[pick_text.index("(")+1 : pick_text.index(")")]
                
                if pick_class == "correct":
                    data.append([week, i, person, int(pts), 0])
                else:
                    data.append([week, i, person, 0, int(pts)])

                i += 1
            person += 1
    
    df = pd.DataFrame(data, columns=["week", "game", "person", "correct_pts", "incorrect_pts"])

    return df

def get_statistics(df):
    
    df["average_pick"] = round((df["correct_pts"] - df["incorrect_pts"]) / df["count"], 2)
    df["average_score"] = round(df["correct_pts"] / df["count"], 2)
    

def raw_to_df(week):
    raw_data = html_to_df(week)
    grouped = raw_data[["game", "correct_pts", "incorrect_pts"]].groupby(["game"])
    res = grouped.sum()
    res["count"] = grouped.game.count()
    get_statistics(res)
    res = res.reset_index()
    res.loc[:, "week"] = week
    res = res.set_index(["week", "game"])
    return res

def send_to_csv(df, path):
    df.to_csv(path)
            

In [58]:
week1 = raw_to_df(1)
week2 = raw_to_df(2)
week3 = raw_to_df(3)
final_df = pd.concat([week1, week2, week3])
final_df.head(5)



correct_pts  incorrect_pts  count  average_pick  average_score
week game                                                                
1    0             117             65     34          1.53           3.44
     1             497              0     34         14.62          14.62
     2              18            223     34         -6.03           0.53
     3               0            446     34        -13.12           0.00
     4             174             45     34          3.79           5.12

In [59]:
send_to_csv(final_df, "data/picking_data.csv")